<a href="https://colab.research.google.com/github/ukf-matusklestinec/Strojovy-preklad-DP/blob/main/Predspracovanie_dat_Trenovanie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# KORPUSY
# https://opus.nlpl.eu/Europarl/en&sk/v7/Europarl



# https://github.com/google/sentencepiece - SentencePiece
# https://opennmt.net/OpenNMT-tf/configuration.html - dokumentácia pre config

In [1]:
!pip install sentencepiece
!pip3 install OpenNMT-py
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.8/262.8 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.9/35.9 MB 44.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 86.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 95.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 110.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.7/227.7 kB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 12.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993225 sha256=d751170cbea012cdcfa2de680b7a432221c68586f16e91394b

In [2]:
import pandas as pd
import numpy as np
import csv
import sentencepiece as spm
import time
from langdetect import detect, LangDetectException  # pre identifikáciu jazykov
import unicodedata  # pre normalizáciu unicode
import torch
import sacrebleu

In [3]:
en_parl = "/content/drive/MyDrive/Colab_Notebooks/Diplomovka/Dáta/europarl/europarl.sk-en.en"
sk_parl = "/content/drive/MyDrive/Colab_Notebooks/Diplomovka/Dáta/europarl/europarl.sk-en.sk"

## Filtrovanie/Čistenie korpusu

In [ ]:
def print_status(operacia, pocet_riadkov, pocet_odst_riadkov, prejdeny_cas):
    print(f"{operacia} Riadky: {pocet_riadkov} | Odstránené riadky: {pocet_odst_riadkov}, Čas: {prejdeny_cas:.2f} sekúnd")

start_time = time.time()

df_source = pd.read_csv(en_parl, names=['Source'], sep="\0", quoting=csv.QUOTE_NONE, skip_blank_lines=False) # dataset pre anglický text (source)
df_target = pd.read_csv(sk_parl, names=['Target'], sep="\0", quoting=csv.QUOTE_NONE, skip_blank_lines=False) # dataset pre slovenský text (target)
df = pd.concat([df_source, df_target], axis=1)
print("Rozmery datasetu (riadky, stĺpce):", df.shape)

# aby fungovala normalizácia
df['Source'] = df['Source'].astype(str)
df['Target'] = df['Target'].astype(str)

# Unicode normalizácia
df['Source'] = df['Source'].apply(lambda x: unicodedata.normalize('NFKC', x))
df['Target'] = df['Target'].apply(lambda x: unicodedata.normalize('NFKC', x))
print_status("Normalizácia na základe UNICODE znakov\t\t\t        ", df.shape[0], 0, time.time() - start_time)


# Odstránenie NaN hodnôt
pocet_riadkov = df.shape[0]
df = df.dropna()
odstranene_riadky = pocet_riadkov - df.shape[0]
print_status("Riadky bez NaN hodnôt\t\t\t\t\t        ", df.shape[0], odstranene_riadky, time.time() - start_time)


# Odstránenie riadkov kde sa riadok source rovná riadku target
pocet_riadkov = df.shape[0]
df = df[df['Source'] != df['Target']]
odstranene_riadky = pocet_riadkov - df.shape[0]
print_status("Odstránenie riadkov, kde sa riadok source rovná riadku target\t", df.shape[0], odstranene_riadky, time.time() - start_time)


# Odstránenie duplikátov
pocet_riadkov = df.shape[0]
df = df.drop_duplicates()
odstranene_riadky = pocet_riadkov - df.shape[0]
print_status("Odstránenie duplicitných/rovnakých riadkov\t\t\t", df.shape[0], odstranene_riadky, time.time() - start_time)


# Odstránenie v dôsledku príliš dlhých viet source/target
# Dĺžka, ktorá sa považuje za "príliš dlhú" záleží od jazyka
pocet_riadkov = df.shape[0]
max_sentence_length = 175
df = df[~((df['Source'].str.count(' ') > max_sentence_length) | (df['Target'].str.count(' ') > max_sentence_length))]
odstranene_riadky = pocet_riadkov - df.shape[0]
print_status("Odstránenie príliš dlhých viet source/target\t\t        ", df.shape[0], odstranene_riadky, time.time() - start_time)

# Odstránenie stopwords
#stop_words_en = STOP_WORDS_EN
#stop_words_sk = STOP_WORDS_SK
#df['Source'] = df['Source'].apply(lambda x: ' '.join([w for w in x.split() if w not in stop_words_en]))
#df['Target'] = df['Target'].apply(lambda x: ' '.join([w for w in x.split() if w not in stop_words_sk]))
#odstranene_riadky = 0
#print_status("Odstránenie stopwords\t\t\t\t\t        ", df.shape[0], odstranene_riadky, time.time() - start_time)

# Stemming
#stemmer = PorterStemmer()
#df['Source'] = df['Source'].apply(lambda x: ' '.join([stemmer.stem(w) for w in x.split()]))
#df['Target'] = df['Target'].apply(lambda x: ' '.join([stemmer.stem(w) for w in x.split()]))
#print_status("Stemming\t\t\t\t\t\t\t        ", df.shape[0], odstranene_riadky, time.time() - start_time)

# konverzia špeciálnych znakov
pocet_riadkov = df.shape[0]

df['Source'] = df['Source'].str.replace(r'&amp;', '&', regex=True)
df['Source'] = df['Source'].str.replace(r'&quot;', '"', regex=True)
df['Source'] = df['Source'].str.replace(r'&apos;', '\'', regex=True)
df['Source'] = df['Source'].str.replace(r'&lt;', '<', regex=True)
df['Source'] = df['Source'].str.replace(r'&gt;', '>', regex=True)

df['Target'] = df['Target'].str.replace(r'&amp;', '&', regex=True)
df['Target'] = df['Target'].str.replace(r'&quot;', '"', regex=True)
df['Target'] = df['Target'].str.replace(r'&apos;', '\'', regex=True)
df['Target'] = df['Target'].str.replace(r'&lt;', '<', regex=True)
df['Target'] = df['Target'].str.replace(r'&gt;', '>', regex=True)
print_status("Konverzia špeciálnych znakov\t\t\t        	", df.shape[0], odstranene_riadky, time.time() - start_time)

# Odstránenie znakov, ktoré nie sú písmená/čísla
df['Source'] = df['Source'].str.replace('[^\w\s]', '', regex=True)
df['Target'] = df['Target'].str.replace('[^\w\s]', '', regex=True)
odstranene_riadky = pocet_riadkov - df.shape[0]
print_status("Odstránenie znakov, ktoré nie sú písmená/čísla\t\t\t", df.shape[0], odstranene_riadky, time.time() - start_time)


# konverzia na malé písmená
pocet_riadkov = df.shape[0]
df['Source'] = df['Source'].str.lower()
df['Target'] = df['Target'].str.lower()
prejdeny_cas = time.time() - start_time
odstranene_riadky = pocet_riadkov - df.shape[0] # tu je to len, aby to pasovalo pre funkciu print_status
print_status("Riadky sú konvertované na malé písmená (lowercase)\t\t", df.shape[0], odstranene_riadky, time.time() - start_time)


# Nahradenie prázdnych riadkov NaN
df = df.replace(r'^\s*$', np.nan, regex=True)

# Dodatočné odstránenie riadkov, kde je NaN, ktoré mohli vzniknúť v dôsledku minulých krokov
pocet_riadkov = df.shape[0]
df = df.dropna()
odstranene_riadky = pocet_riadkov - df.shape[0]
print_status("Riadky bez NaN hodnôt\t\t\t\t\t        ", df.shape[0], odstranene_riadky, time.time() - start_time)

# kontrola jazyka - tento proces trvá najdlhšie
def safe_detect(text):
    try:
        return detect(text)
    except LangDetectException:
        return None

pocet_riadkov = df.shape[0]
df['source_jazyk'] = df['Source'].apply(lambda x: safe_detect(x) if x and len(x.split()) > 1 else None)
df['target_jazyk'] = df['Target'].apply(lambda x: safe_detect(x) if x and len(x.split()) > 1 else None)

df = df[(df['source_jazyk'] == "en") & (df['target_jazyk'] == "sk")]
df.drop(columns=['source_jazyk', 'target_jazyk'], inplace=True)
odstranene_riadky = pocet_riadkov - df.shape[0]
print_status("Kontrola jazykov\t\t\t\t\t\t        ", df.shape[0], odstranene_riadky, time.time() - start_time)


# Zmenenie poradia riadkov
df = df.sample(frac=1, random_state=42).reset_index(drop=True) # pridanie random_state, aby sme dostali vždy rovnako zamiešaný dataset
print("Prehodenie poradia riadkov\t\t\t\t\t Riadky:", df.shape[0])

prejdeny_cas = time.time() - start_time
print(f"Celkový prejdený čas: {prejdeny_cas:.2f} sekúnd")

source_subor = en_parl+'-filtered.'+ "en"
target_subor = sk_parl+'-filtered.'+ "sk"

df_source_en = df["Source"]
df_target_sk = df["Target"]

df_source_en.to_csv(source_subor, header=False, index=False, quoting=csv.QUOTE_NONE, sep="\n")
print("Source súbor uložený:", source_subor)
df_target_sk.to_csv(target_subor, header=False, index=False, quoting=csv.QUOTE_NONE, sep="\n")
print("Target súbor uložený:", target_subor)

Rozmery datasetu (riadky, stĺpce): (640715, 2)
Normalizácia na základe UNICODE znakov			         Riadky: 640715 | Odstránené riadky: 0, Čas: 6.29 sekúnd
Riadky bez NaN hodnôt					         Riadky: 640715 | Odstránené riadky: 0, Čas: 6.74 sekúnd
Odstránenie riadkov, kde sa riadok source rovná riadku target	 Riadky: 637465 | Odstránené riadky: 3250, Čas: 7.00 sekúnd
Odstránenie duplicitných/rovnakých riadkov			 Riadky: 621617 | Odstránené riadky: 15848, Čas: 8.33 sekúnd
Odstránenie príliš dlhých viet source/target		         Riadky: 621601 | Odstránené riadky: 16, Čas: 11.44 sekúnd
Konverzia špeciálnych znakov			        	 Riadky: 621601 | Odstránené riadky: 16, Čas: 16.41 sekúnd
Odstránenie znakov, ktoré nie sú písmená/čísla			 Riadky: 621601 | Odstránené riadky: 0, Čas: 21.50 sekúnd
Riadky sú konvertované na malé písmená (lowercase)		 Riadky: 621601 | Odstránené riadky: 0, Čas: 22.85 sekúnd
Riadky bez NaN hodnôt					         Riadky: 621597 | Odstránené riadky: 4, Čas: 26.51 sekúnd
Kontrol

## Subwording

In [4]:
en_filt = r"/content/drive/MyDrive/Colab_Notebooks/Diplomovka/Dáta/europarl/europarl.sk-en.en-filtered.en"
sk_filt = r"/content/drive/MyDrive/Colab_Notebooks/Diplomovka/Dáta/europarl/europarl.sk-en.sk-filtered.sk"

In [5]:
#https://github.com/google/sentencepiece


#Parametre
vocab_size = 8000
character_coverage = 0.9995

# pre prípad veľkých korpusov -> --train_extremely_large_corpus=false

# Model pre subwordovanie anglického textu (source)
source = f'--input={en_filt} --model_prefix=source --vocab_size={vocab_size} --hard_vocab_limit=false --character_coverage={character_coverage} --split_digits=true --model_type=unigram'
spm.SentencePieceTrainer.train(source)
print("Trénovanie SentencePiece modelu bolo pre source súbor dokončené")


# Model pre subwordovanie slovenského textu (target)
target = f'--input={sk_filt} --model_prefix=target --vocab_size={vocab_size} --hard_vocab_limit=false --character_coverage={character_coverage} --split_digits=true --model_type=unigram'
spm.SentencePieceTrainer.train(target)
print("Trénovanie SentencePiece modelu bolo pre target súbor dokončené")

Trénovanie SentencePiece modelu bolo pre source súbor dokončené
Trénovanie SentencePiece modelu bolo pre target súbor dokončené


In [6]:
source_model = r"/content/source.model"
target_model = r"/content/target.model"
source_subworded = "/content/drive/MyDrive/Colab_Notebooks/Diplomovka/Dáta/europarl/source_subworded"
target_subworded = "/content/drive/MyDrive/Colab_Notebooks/Diplomovka/Dáta/europarl/target_subworded"

In [7]:
def subwordovanie(sp, vstup, vystup, interval=100000):
    try:
        with open(vstup, encoding="utf-8") as vstup, open(vystup, "w+", encoding="utf-8") as vystup:
            for i, line in enumerate(vstup):
                line = line.strip()
                tokens = sp.encode_as_pieces(line)
                vystup.write(" ".join(tokens) + "\n")

                if (i + 1) % interval == 0:
                    print(f"Spracovaných {i + 1} riadkov v {vstup}")
    except Exception as e:
        print(f"Nastala chyba v procese {vstup}: {e}")

sp = spm.SentencePieceProcessor()

sp.load(source_model)
subwordovanie(sp, en_filt, source_subworded)
print(f"Subwordovanie Source súboru bolo dokončené: {source_subworded}")

sp.load(target_model)
subwordovanie(sp, sk_filt, target_subworded)
print(f"Subwordovanie Target súboru bolo dokončené: {target_subworded}")

Spracovaných 100000 riadkov v /content/drive/MyDrive/Colab_Notebooks/Diplomovka/Dáta/europarl/europarl.sk-en.en-filtered.en
Spracovaných 200000 riadkov v /content/drive/MyDrive/Colab_Notebooks/Diplomovka/Dáta/europarl/europarl.sk-en.en-filtered.en
Spracovaných 300000 riadkov v /content/drive/MyDrive/Colab_Notebooks/Diplomovka/Dáta/europarl/europarl.sk-en.en-filtered.en
Spracovaných 400000 riadkov v /content/drive/MyDrive/Colab_Notebooks/Diplomovka/Dáta/europarl/europarl.sk-en.en-filtered.en
Spracovaných 500000 riadkov v /content/drive/MyDrive/Colab_Notebooks/Diplomovka/Dáta/europarl/europarl.sk-en.en-filtered.en
Spracovaných 600000 riadkov v /content/drive/MyDrive/Colab_Notebooks/Diplomovka/Dáta/europarl/europarl.sk-en.en-filtered.en
Subwordovanie Source súboru bolo dokončené: /content/drive/MyDrive/Colab_Notebooks/Diplomovka/Dáta/europarl/source_subworded
Spracovaných 100000 riadkov v /content/drive/MyDrive/Colab_Notebooks/Diplomovka/Dáta/europarl/europarl.sk-en.sk-filtered.sk

## Výpis riadkov súboru

In [8]:
with open(source_subworded, "r", encoding="utf-8") as file:
    for i, line in enumerate(file):
        print(line.strip())
        if i >= 5:
            break

print()

with open(target_subworded, "r", encoding="utf-8") as file:
    for i, line in enumerate(file):
        print(line.strip())
        if i >= 5:
            break

▁i ▁would ▁refer ▁in ▁particular ▁to ▁green ▁public ▁procurement s ▁which ▁will ▁allow ▁public ▁sector ▁agencies ▁to ▁play ▁a ▁leading ▁role ▁in ▁saving ▁energy ▁by ▁making ▁use ▁of ▁the ▁new ▁technological ▁applications ▁of ▁ict
▁the ▁presence ▁of ▁origin ▁marking ▁would ▁surely ▁have ▁provided ▁a ▁guarantee ▁of ▁product ▁quality ▁and ▁protection
▁the ▁open ▁scope ▁of ▁the ▁legislation ▁as ▁recommended ▁by ▁the ▁environment ▁committee ▁is ▁a ▁much ▁better ▁approach ▁than ▁that ▁proposed ▁by ▁the ▁commission
▁of ▁course ▁this ▁means ▁that ▁if ▁one ▁million ▁european ▁citizens ▁wish ▁to ▁dispens e ▁with ▁the ▁strasbourg ▁site ▁that ▁desire ▁too ▁must ▁be ▁he ed ed
▁indeed ▁the ▁eu ▁often ▁does ▁not ▁appear ▁to ▁know ▁who ▁it ▁is ▁actually ▁funding ▁and ▁who ▁is ▁pull ing ▁its ▁ str ing s
▁women ▁are ▁the ▁favour it e ▁victims ▁of ▁the ▁recession ▁due ▁to ▁redundancies ▁primari ly ▁affect ing ▁precarious ▁jobs

▁rád ▁by ▁som ▁spomenul ▁najmä ▁ekologické ▁verejné ▁obstarávanie ▁ktoré ▁age

##Rozdelenie dát na množiny

Dáta rozdelujeme do 3 častí:

    trénovací dataset - používa sa na trénovanie modelu;
    development dataset - používa sa na validáciu modelu počas trénovania, aby sa mohli vylepšiť parametre modelu
    testovací dataset - používa sa na konečné vyhodotenie (evaluáciu) modelu na modelom nevidených dátach

In [9]:
source_subor = r"/content/drive/MyDrive/Colab_Notebooks/Diplomovka/Dáta/europarl/source_subworded"
target_subor = r"/content/drive/MyDrive/Colab_Notebooks/Diplomovka/Dáta/europarl/target_subworded"

In [10]:
np.random.seed(42)


# Veľkosť súborov pre dev a test text
data_size = 4000

development = data_size
test = data_size


def train_test_dev(development, test, source_subor, target_subor):
    df_source = pd.read_csv(source_subor, names=['Source'], sep="\0", quoting=csv.QUOTE_NONE, skip_blank_lines=False, encoding="utf-8")
    df_target = pd.read_csv(target_subor, names=['Target'], sep="\0", quoting=csv.QUOTE_NONE, skip_blank_lines=False, encoding="utf-8")
    df = pd.concat([df_source, df_target], axis=1)

    # Odstránenie prázdnych riadkov
    df.dropna(inplace=True)

    # Zamiešanie datasetu
    df = df.sample(frac=1, random_state=42).reset_index(drop=True) # pridanie random_state, aby sme dostali vždy rovnako zamiešaný dataset

    # Rozdelenie datasetu
    df_dev = df[:development]
    df_test = df[development:development + test]
    df_train = df[development + test:]

    def vytvorenie_suboru(df, source_subor, target_subor):
        df['Source'].to_csv(source_subor, index=False, header=False, quoting=csv.QUOTE_NONE, sep=',')
        df['Target'].to_csv(target_subor, index=False, header=False, quoting=csv.QUOTE_NONE, sep=',')


    vytvorenie_suboru(df_train, source_subor + '.train', target_subor + '.train')
    vytvorenie_suboru(df_dev, source_subor + '.dev', target_subor + '.dev')
    vytvorenie_suboru(df_test, source_subor + '.test', target_subor + '.test')


train_test_dev(development, test, source_subor, target_subor)


In [11]:
source_test = r"/content/drive/MyDrive/Colab_Notebooks/Diplomovka/Dáta/europarl/source_subworded.test"
target_test = r"/content/drive/MyDrive/Colab_Notebooks/Diplomovka/Dáta/europarl/target_subworded.test"

In [12]:
with open(source_test, "r", encoding="utf-8") as file:
    for i, line in enumerate(file):
        print(line.strip())
        if i >= 5:
            break

print()

with open(target_test, "r", encoding="utf-8") as file:
    for i, line in enumerate(file):
        print(line.strip())
        if i >= 5:
            break

▁firstly ▁there ▁are ▁insufficient ▁organs ▁available ▁in ▁the ▁eu
▁there ▁is ▁poverty ▁in ▁the ▁world
▁the ▁european ▁union ▁does ▁indeed ▁require ▁ambition ▁to ▁defeat ▁the ▁challenges ▁presented ▁by ▁the ▁crisis ▁but ▁that ▁ambition ▁must ▁not ▁cause ▁problems ▁for ▁the ▁effort ▁at ▁budget ary ▁consolidation ▁which ▁is ▁being ▁demanded ▁of ▁member ▁states ▁in ▁view ▁of ▁the ▁weakness ▁of ▁their ▁public ▁accounts ▁and ▁their ▁excessive ▁deficits
▁however ▁the ▁different ▁implementation ▁methods ▁of ▁member ▁states ▁have ▁made ▁it ▁difficult ▁for ▁the ▁service ▁sector ▁to ▁take ▁full ▁advantage ▁of ▁the ▁directive
▁i ▁voted ▁for ▁the ▁report ▁on ▁the ▁demographic ▁challenge ▁and ▁solidarity ▁between ▁generations ▁because ▁under ▁the ▁current ▁circumstances ▁of ▁the ▁unprecedented ▁demographic ▁problems ▁posed ▁by ▁increased ▁life ▁expectancy ▁and ▁falling ▁birth ▁rates ▁they ▁constitute ▁one ▁of ▁the ▁most ▁urgent ▁social ▁policy ▁challenges ▁of ▁coming ▁years
▁with ▁a ▁fixed ▁nationa

In [13]:
config = '''

seed: 42 # aby sme dosiahli rovnakých výsledkov a pre lepšie porovnávanie medzi modelmi

save_data: vocab  # zložka, kde sa uloží slovná zásoba vygenerovaná pomocou onmt_build_vocab

data:
    corpus:
        # súbory určené na trénovanie
        path_src: /content/drive/MyDrive/Colab_Notebooks/Diplomovka/Dáta/europarl/source_subworded.train
        path_tgt: /content/drive/MyDrive/Colab_Notebooks/Diplomovka/Dáta/europarl/target_subworded.train
        transforms: [filtertoolong]
        weight: 1
    valid:
        # súbory určené na validáciu
        path_src: /content/drive/MyDrive/Colab_Notebooks/Diplomovka/Dáta/europarl/source_subworded.dev
        path_tgt: /content/drive/MyDrive/Colab_Notebooks/Diplomovka/Dáta/europarl/target_subworded.dev
        transforms: [filtertoolong]

# súbory, v ktorých je uložená slovná zásoba pre source (anglický jazyk) a target (slovenský jazyk) onmt_build_vocab
src_vocab: vocab/source.vocab
tgt_vocab: vocab/target.vocab

# veľkosť slovnej zásoby - hodnoty by mali byť rovnaké ako v sentencepiece
src_vocab_size: 8000
tgt_vocab_size: 8000

src_subword_model: /content/source.model
tgt_subword_model: /content/target.model

# Miesto uloženia súborov
log_file:   /content/drive/MyDrive/Colab_Notebooks/Diplomovka/Dáta/europarl/train.log
save_model: /content/drive/MyDrive/Colab_Notebooks/Diplomovka/Dáta/europarl/complete_model

train_steps: 100000 # počet krokov trénovania
valid_steps: 10000  # po n krokoch vykonať validáciu pomocou dev setu
warmup_steps: 8000 # počet by mal byť priamoúmerný počtu krokov trénovania
early_stopping: 4 # ak sa model nezlepší po n validáciach, trénovanie zastaví
save_checkpoint_steps: 5000   # uloženie modelu po n krokoch
keep_checkpoint: 10  # počet posledných n uložených modelov
report_every: 500 # výpis informácií o trénovaní po n-krokoch

decoder_type: transformer
encoder_type: transformer
word_vec_size: 512   # dimenzionálnosť word embeddingu
hidden_size: 512     # dimenzionálnosť skrytých stavov v enkódery a dekódery
layers: 6             # počet vrstiev v enkódery a dekódery   - alternatíva je enc_layers a dec_layers
transformer_ff: 2048  # dimenzionálnosť doprednej vrstvy v transformery
heads: 8              # počet attention head v multi-head-attention

accum_count: 2 # počet n krokov, po ktorých sa aktualizujú parametre
optim: adam
adam_beta1: 0.9
adam_beta2: 0.998
decay_method: noam
learning_rate: 2.0    # miera učenia
max_grad_norm: 0.0    # gradient clipping, pomáha, aby v modely nenastal explodujúci gradient (nemožnosť modelu sa učiť v dôsledku vysokej miery učenia), emôže pomôcť pre lepšiu stabilitu modelu

param_init: 0.0
param_init_glorot: 'true'
position_encoding: 'true'

batch_size: 1024
valid_batch_size: 1024
batch_type: tokens
normalization: tokens
max_generator_batches: 2

dropout: 0.1           # regularizácia, ktorá vytvára šancu (0.1 = 10%) na "vypnutie" nodu vo vrstve behom trénovania, pomáha proti overfittingu
label_smoothing: 0.1

world_size: 1
gpu_ranks: 0

# INFO
log_file: train.log
log_level: INFO

'''

with open("config.yaml", "w+", encoding="utf-8") as config_yaml:
  config_yaml.write(config)

In [14]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-2ad78e68-7908-2501-57d8-2e092505cd05)


Použíjeme Framework OpenNMT

In [ ]:
# Vytvorenie vocabulary
!onmt_build_vocab -config config.yaml -n_sample -1

Corpus corpus_1's weight should be given. We default it to 1 for you.
[2024-02-24 13:56:54,917 INFO] Counter vocab from -1 samples.
[2024-02-24 13:56:54,917 INFO] n_sample=-1: Build vocab on full datasets.
[2024-02-24 13:57:25,943 INFO] * Transform statistics for corpus_1(100.00%):
			* FilterTooLongStats(filtered=26)

[2024-02-24 13:57:25,965 INFO] Counters src: 8070
[2024-02-24 13:57:25,965 INFO] Counters tgt: 8052


In [ ]:
# Trénovanie the NMT model
!onmt_train -config config.yaml -gpu_ranks 0

## Preklad

In [ ]:
config_translate = '''

model: /content/drive/MyDrive/Colab_Notebooks/Diplomovka/Dáta/europarl/complete_model_step_35000.pt     # sem vložiť najlepší model trénovania
src: /content/drive/MyDrive/Colab_Notebooks/Diplomovka/Dáta/europarl/source_subworded.test
output: /content/drive/MyDrive/Colab_Notebooks/Diplomovka/Dáta/europarl/translated

log_file: translate.log

seed: 42

beam_size: 4
min_length: 1
length_penalty: avg

batch_size: 32
gpu: 0

coverage_penalty: summary
beta: 0.5

'''

with open("config_translated.yaml", "w+", encoding="utf-8") as config_yaml:
  config_yaml.write(config_translate)

In [ ]:
# Preklad
!onmt_translate -config config_translated.yaml

[2024-02-25 10:54:14,376 INFO] Loading checkpoint from /content/drive/MyDrive/Colab_Notebooks/Diplomovka/Dáta/europarl/complete_model_step_35000.pt
[2024-02-25 10:54:27,975 INFO] Loading data into the model
[2024-02-25 10:55:22,192 INFO] PRED SCORE: -4.8122, PRED PPL: 123.01 NB SENTENCES: 4000
Time w/o python interpreter load/terminate:  68.28334426879883


In [ ]:
translated = r"/content/drive/MyDrive/Colab_Notebooks/Diplomovka/Dáta/europarl/translated"
with open(translated, "r", encoding="utf-8") as file:
    for i, line in enumerate(file):
        print(line.strip())
        if i >= 5:
            break

print()

▁v ▁súčasnej ▁situácii ▁by ▁mala ▁európska ▁únia ▁venovať ▁osobitnú ▁pozornosť ▁svojim ▁ 2 0 ▁milió nom ▁malých ▁a ▁stredných ▁podnikov
▁takéto ▁opatrenie ▁nám ▁umožní ▁zab iť ▁niekoľko ▁v tá kov ▁jednou ▁r anou ▁je ▁to ▁účinný ▁nástroj ▁proti ▁špeku l áciám ▁a ▁príjmy ▁nám ▁umožnia ▁riešiť ▁vlád ne ▁defici ty ▁a ▁financovanie ▁naliehav ých ▁sociálnych ▁projektov ▁životného ▁prostredia ▁rozvojovej ▁pomoci ▁projektov ▁infraštruktúry ▁atď
▁tento ▁návrh ▁referen čného ▁rámca ▁nie ▁je ▁hroz bou ▁pre ▁spôsob ▁akým ▁členské ▁štáty ▁organiz ujú ▁odborné ▁vzdelávanie ▁v ▁európe ▁ale ▁môžeme ▁stav ať ▁na ▁tradí cii ▁ktorú ▁máme ▁bez ▁ohľadu ▁na ▁to ▁či ▁je ▁organiz ovaná ▁na ▁miestnej ▁regionálnej ▁alebo ▁národnej ▁úrovni
▁v ▁týchto ▁veciach ▁sa ▁toho ▁veľa ▁hod ilo ▁takže ▁zač ni me ▁pripomenu tím ▁kľúčových ▁fak tov
▁toto ▁očakávajú ▁občania ▁spotrebitelia ▁od ▁parlamentu ▁aj ▁rady
▁v ▁tejto ▁súvislosti ▁podporujem ▁pozmeňujúci ▁a ▁doplňujúci ▁návrh ▁ 2 ▁predložený ▁skupinou ▁zelených ▁ktorý 

In [ ]:
target_model = r"/content/target.model"
translation = r"/content/drive/MyDrive/Colab_Notebooks/Diplomovka/Dáta/europarl/translated"
translation_desubword = translation + ".desubword"

sp = spm.SentencePieceProcessor()
sp.load(target_model)

# Desubword prekladový súbor
with open(translation, encoding='utf-8') as vstup, open(translation, "w", encoding='utf-8') as vystup:
    for line in vstup:
        desub = sp.decode_pieces(line.strip().split(" "))
        vystup.write(desub + "\n")

print(f"Dekódovanie bolo dokončené! Výstupný súbor: {translation_desubword}")

target_test = r"/content/drive/MyDrive/Colab_Notebooks/Diplomovka/Dáta/europarl/target_subworded.test"
target_test_desubword = target_test + ".desubword"

with open(target_test, encoding='utf-8') as vstup, open(target_test_desubword, "w", encoding='utf-8') as vystup:
    for line in vstup:
        desub = sp.decode_pieces(line.strip().split(" "))
        vystup.write(desub + "\n")

print(f"Dekódovanie bolo dokončené! Výstupný súbor: {target_test_desubword}")

In [ ]:
reference = r"/content/drive/MyDrive/Colab_Notebooks/Diplomovka/Dáta/europarl/target_subworded.test.desubword"
preklad = r"/content/drive/MyDrive/Colab_Notebooks/Diplomovka/Dáta/europarl/translated.desubword"

In [ ]:
with open(reference, "r", encoding="utf-8") as file:
    for i, line in enumerate(file):
        print(line.strip())
        if i >= 5:
            break

print()

v súčasnej situácii by mala európska únia osobitnú pozornosť venovať svojim 20 miliónom malých a stredných podnikov
takéto opatrenie nám umožní zabiť niekoľko múch jednou ranou je účinným nástrojom proti špekuláciám a príjmy nám umožnia poradiť si so štátnymi deficitmi a financovaním naliehavých sociálnych projektov životné prostredie rozvojová pomoc infraštruktúrne projekty atď
tento návrh na referenčný rámec neohrozuje spôsob organizovania odborného vzdelávania v členských štátoch v celej európe môžeme však budovať na tradícii ktorú máme bez ohľadu na to či je organizované na miestnej regionálnej alebo vnútroštátnej úrovni
na písanie o týchto problémoch sa už vyplytvalo veľa atramentu začneme teda pripomenutím základných faktov
to od parlamentu a rady očakávajú občania spotrebitelia
v súvislosti s tým podporujem pozmeňujúci a doplňujúci návrh č 2 ktorý predložila skupina zelených a ktorý si vyžaduje podstatné posilnenie regulačného a dozorného rámca eú s cieľom zachovať finančnú stab

In [ ]:
with open(preklad, "r", encoding="utf-8") as file:
    for i, line in enumerate(file):
        print(line.strip())
        if i >= 5:
            break

print()

v súčasnej situácii by mala európska únia venovať osobitnú pozornosť svojim 20 miliónom malých a stredných podnikov
takéto opatrenie nám umožní zabiť niekoľko vtákov jednou ranou je to účinný nástroj proti špekuláciám a príjmy nám umožnia riešiť vládne deficity a financovanie naliehavých sociálnych projektov životného prostredia rozvojovej pomoci projektov infraštruktúry atď
tento návrh referenčného rámca nie je hrozbou pre spôsob akým členské štáty organizujú odborné vzdelávanie v európe ale môžeme stavať na tradícii ktorú máme bez ohľadu na to či je organizovaná na miestnej regionálnej alebo národnej úrovni
v týchto veciach sa toho veľa hodilo takže začnime pripomenutím kľúčových faktov
toto očakávajú občania spotrebitelia od parlamentu aj rady
v tejto súvislosti podporujem pozmeňujúci a doplňujúci návrh 2 predložený skupinou zelených ktorý žiada podstatné posilnenie európskeho regulačného a dozorného rámca s cieľom udržať finančnú stabilitu



In [ ]:
#!pip install spacy

In [ ]:
#!python -m spacy download en_core_web_sm
!python -m spacy download xx_ent_wiki_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 41.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('xx_ent_wiki_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import spacy
import sacrebleu
from nltk.translate import meteor_score as ms
import nltk
nltk.download('wordnet')

nlp = spacy.load("xx_ent_wiki_sm") #https://github.com/explosion/spacy-models/releases/tag/xx_ent_wiki_sm-3.7.0

[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
reference = r"/content/drive/MyDrive/Colab_Notebooks/Diplomovka/Dáta/europarl/target_subworded.test.desubword"
preklad = r"/content/drive/MyDrive/Colab_Notebooks/Diplomovka/Dáta/europarl/translated.desubword"

In [ ]:
# referenčné hodnoty
refs = []

with open(reference, encoding='utf-8') as reference:
    for line in reference:
        line = line.strip()
        refs.append(line)

print("Referenčná veta v SK jazyku:", refs[0]) # výpis prvej referenčenj vety, pomocou ktorej budeme porovnávať vety

refs = [refs]  # list listov, teda forma, ktorú si vyžaduje SacreBLEU

# hodnoty, ktoré sme preložili
preds = []

with open(preklad, encoding='utf-8') as prek:
    for line in prek:
        line = line.strip()
        preds.append(line)

print("Preložená veta:\t\t    ", preds[0]) # výpis prvého preloženého riadka

# BLEU skóre - výpočet BLEU na základe porovnania referenčných viet s preloženými vetami
bleu = sacrebleu.corpus_bleu(preds, refs)
print("BLEU:  ", bleu.score)

# tokenizácia je potrebná pre rozdelenie viet do menších častí (tokenov), aby Meteor dokázal porovnávať slová a ich význam
def tokenizacia(text):
    doc = nlp(text)
    return [token.text for token in doc]

meteor_scores = []
for refs_segment, pred in zip(refs, preds):
    ref = refs_segment[0]
    token_ref = tokenizacia(ref) # tokenizácia referenčných viet
    token_pred = tokenizacia(pred) # tokenizáci preložených viet

    # Výpočet METEOR skóre - meteor skóre narozdiel od bleu zohľadnuje sémantiku do istej miery
    score = ms.meteor_score([token_ref], token_pred) # výpočet meteor skóre pre jednu vetu
    meteor_scores.append(score)

average_meteor_score = sum(meteor_scores) / len(meteor_scores) # výpočet priemeru Meteor skóre, pre celú množinu preložených viet
print("METEOR: ", average_meteor_score)

Referenčná veta v SK jazyku: v súčasnej situácii by mala európska únia osobitnú pozornosť venovať svojim 20 miliónom malých a stredných podnikov
Preložená veta:		     v súčasnej situácii by mala európska únia venovať osobitnú pozornosť svojim 20 miliónom malých a stredných podnikov
BLEU:   35.86040111007451
METEOR:  0.9934866680236109


In [ ]:
token_ref

['v',
 'súčasnej',
 'situácii',
 'by',
 'mala',
 'európska',
 'únia',
 'osobitnú',
 'pozornosť',
 'venovať',
 'svojim',
 '20',
 'miliónom',
 'malých',
 'a',
 'stredných',
 'podnikov']

In [ ]:
token_pred

['v',
 'súčasnej',
 'situácii',
 'by',
 'mala',
 'európska',
 'únia',
 'venovať',
 'osobitnú',
 'pozornosť',
 'svojim',
 '20',
 'miliónom',
 'malých',
 'a',
 'stredných',
 'podnikov']

In [ ]:
from google.colab import runtime
runtime.unassign()